In [1]:
import re
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import datetime as dt
import sqlite3

In [2]:
'''scrapes game url returning a dict'''
url = 'https://www.pro-football-reference.com/boxscores/202012130car.htm'
res = requests.get(url)
# use re to escape html comments
comm = re.compile("<!--|-->")
soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
# get content div, holds all body content
divs = soup.findAll('div', id = "content")
# table for points by quarter
points = pd.read_html(res.text)[0]
# table for scoring stats
stats = pd.read_html(str(divs[0].findAll(
    "div",id=re.compile("^all_team_stats"))[0].findAll('table')[0]))[0]

# get datetime object for the dict
date_ = divs[0].findAll("div", class_='scorebox')[0].findAll(
    'div',class_='scorebox_meta')[0].findAll('div')[0].text
time_ = divs[0].findAll("div", class_='scorebox')[0].findAll(
    'div', class_='scorebox_meta')[0].findAll('div')[1].text.replace('Start Time: ', '')
datestr = date_ + ' ' + time_
dateobj = dt.datetime.strptime(datestr, '%A %b %d, %Y %I:%M%p')

# get week number, all of the playoffs is called week 18
if soup.findAll('h2')[0].text.endswith('Playoffs') == True:
    week_ = '18'
else:
    week_ = soup.findAll('h2')[0].text.replace('NFL Scores — Week ', '')

if dateobj.month >= 7:
    season_ = dateobj.year
else:
    season_ = dateobj.year - 1

game_dict = {}

In [3]:
pd.read_html(res.text)[0]

,Unnamed: 0,Unnamed: 1,1,2,3,4,Final
0,via Sports Logos.net About logos,Denver Broncos,6,7,12,7,32
1,via Sports Logos.net About logos,Carolina Panthers,0,7,3,17,27


In [4]:
stats.iloc[5,1]

'365'

In [5]:
stats

,Unnamed: 0,DEN,CAR
0,First Downs,23,22
1,Rush-Yds-TDs,30-96-0,21-125-3
2,Cmp-Att-Yd-TD-INT,21-27-280-4-0,30-40-283-0-0
3,Sacked-Yards,1-11,4-38
4,Net Pass Yards,269,245
5,Total Yards,365,370
6,Fumbles-Lost,2-1,0-0
7,Turnovers,1,0
8,Penalties-Yards,2-18,7-48
9,Third Down Conv.,3-8,6-14


In [6]:
stats.iloc[3, 2].split('-')[0]

'4'

In [7]:
returns = pd.read_html(str(divs[0].findAll(
    "table",id='returns')))[0]

In [8]:
returns.iloc[0,0]

'Diontae Spencer'

In [9]:
returns.iloc[1,0]

nan

In [10]:
returns.iloc[2,0]

'Player'

In [11]:
import math

In [12]:
split_row = None

In [13]:
try:
    math.isnan(returns.iloc[0,0])
    split_row = 0
except:
    pass
try:
    math.isnan(returns.iloc[1,0])
    split_row = 1
except:
    pass
try:
    math.isnan(returns.iloc[2,0])
    split_row = 2
except:
    pass
try:
    math.isnan(returns.iloc[3,0])
    split_row = 3
except:
    pass
try:
    math.isnan(returns.iloc[4,0])
    split_row = 4
except:
    pass

In [14]:
split_row

1

In [15]:
returns

Unnamed: 0_level_0 Unnamed: 1_level_0  Kick Returns                \
              Player                 Tm            Rt           Yds   
0    Diontae Spencer                DEN             1             5   
1                NaN                NaN  Kick Returns  Kick Returns   
2             Player                 Tm            Rt           Yds   
3      Pharoh Cooper                CAR             0             0   
4     Trenton Cannon                CAR             1            16   

                                             Punt Returns                \
           Y/Rt            TD           Lng           Ret           Yds   
0           5.0             0             5             1            83   
1  Kick Returns  Kick Returns  Kick Returns  Punt Returns  Punt Returns   
2          Y/Rt            TD           Lng           Ret           Yds   
3           NaN             0             0             2             0   
4          16.0             0            16             0             0   

                                             
            Y/R            TD           Lng  
0          83.0             1            83  
1  Punt Returns  Punt Returns  Punt Returns  
2           Y/R            TD           Lng  
3           0.0             0             0  
4           NaN             0             0

In [16]:
returns.columns = returns.columns.droplevel()

In [17]:
away_returns = returns.copy().loc[:split_row-1]

In [18]:
away_returns.columns = ['Player', 'Tm', 'Rt', 'Yds', 'Y/Rt', 'TD-kick', 'Lng', 'Ret', 'Yds', 'Y/R', 'TD-punt', 'Lng']

In [19]:
away_returns['TD-kick'].astype(int).sum() + away_returns['TD-punt'].astype(int).sum()

1

In [20]:
home_returns = returns.copy().loc[split_row+2:]

In [21]:
home_returns.columns = ['Player', 'Tm', 'Rt', 'Yds', 'Y/Rt', 'TD-kick', 'Lng', 'Ret', 'Yds', 'Y/R', 'TD-punt', 'Lng']

In [22]:
home_returns['TD-kick'].astype(int).sum() + home_returns['TD-punt'].astype(int).sum()

0

In [29]:
away_returns

,Player,Tm,Rt,Yds,Y/Rt,TD-kick,Lng,Ret,Yds,Y/R,TD-punt,Lng
0,Diontae Spencer,DEN,1,5,5.0,0,5,1,83,83.0,1,83


In [23]:
returns.loc[split_row+2:]

,Player,Tm,Rt,Yds,Y/Rt,TD,Lng,Ret,Yds,Y/R,TD,Lng
3,Pharoh Cooper,CAR,0,0,NaN,0,0,2,0,0.0,0,0
4,Trenton Cannon,CAR,1,16,16.0,0,16,0,0,NaN,0,0


In [24]:
returns.loc[:split_row-1]

,Player,Tm,Rt,Yds,Y/Rt,TD,Lng,Ret,Yds,Y/R,TD,Lng
0,Diontae Spencer,DEN,1,5,5.0,0,5,1,83,83.0,1,83


In [25]:
away_returns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Player   1 non-null      object
 1   Tm       1 non-null      object
 2   Rt       1 non-null      object
 3   Yds      1 non-null      object
 4   Y/Rt     1 non-null      object
 5   TD-kick  1 non-null      object
 6   Lng      1 non-null      object
 7   Ret      1 non-null      object
 8   Yds      1 non-null      object
 9   Y/R      1 non-null      object
 10  TD-punt  1 non-null      object
 11  Lng      1 non-null      object
dtypes: object(12)
memory usage: 224.0+ bytes


In [26]:
away_returns = returns.loc[:split_row-1]

In [27]:
away_returns.columns = ['Player', 'Tm', 'Rt', 'Yds', 'Y/Rt', 'TD-kick', 'Lng', 'Ret', 'Yds', 'Y/R', 'TD-punt', 'Lng']